In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [ ]:
train = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_train.tsv', sep='\t', header=None, names=['text', 'class'])
test = pd.read_csv('../input/qa-intents-dataset-university-domain/dataset_test.tsv', sep='\t', header=None, names=['text', 'class'])

In [ ]:
train_token = tokenizer(train['text'].tolist(),padding=True, truncation=True, return_tensors='pt')
test_token = tokenizer(test['text'].tolist(),padding=True, truncation=True, return_tensors='pt')

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
with torch.no_grad():
    train_em = mean_pooling(model(**train_token),train_token['attention_mask'])
    test_em = mean_pooling(model(**test_token),test_token['attention_mask'])

In [ ]:
!pip install nmslib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 40.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pybind11
    Found existing installation: pybind11 2.10.3
    Uninstalling pybind11-2.10.3:
      Successfully uninstalled pybind11-2.10.3


In [ ]:
import nmslib
train_index = nmslib.init(method='hnsw', space='cosinesimil')
train_index.addDataPointBatch(train_em, ids = list(range(train_em.shape[0])))
train_index.createIndex({'post': 2}, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
*****************************************************

In [ ]:
for n in np.random.randint(0, test.shape[0], 10):
    ids ,dist = train_index.knnQuery(test_em[n],k=5)
    print(n, test.iloc[n]['text'], test.iloc[n]['class'])
    for i, d in zip(ids, dist):
        print(d, train.iloc[i]['text'], train.iloc[i]['class'])
    print()

383 стоимость интернета расположен где) nsunet_cost
0.016656518 расположен где стоимость интернета) nsunet_cost
0.019819975 цена интернета расположен где) nsunet_cost
0.019819975 цена интернета расположен где) nsunet_cost
0.025439262 стоимость интернета узнать) nsunet_cost
0.037911355 цена интернета где есть) nsunet_cost

756 расписание Осташевская Л.А. sched_teacher
0.0024335384 расписание Скворцова М.А. sched_teacher
0.002450645 расписание Бадальян Л.В. sched_teacher
0.0025234222 расписание Шевчук Н.В. sched_teacher
0.0026016831 расписание для Краснова С.А. sched_teacher
0.0026373267 расписание Литвиненко С.А. sched_teacher

237 сформировать организацию студенческую student_union_create
0.028783143 организация студенческая сформировать student_union_create
0.034049988 создавать организация студенческая student_union_create
0.03518021 организация студенческая создать student_union_create
0.03945923 организация студенческая создавать student_union_create
0.04276228 сформировать объедин